In [1]:
from NCHelper import NCHelper
import numpy as np
import rasterio

In [2]:
date_str = "2023111412"
ds = f"http://182.16.248.173:8080/dods/INA-NWP/{date_str}/{date_str}-d01-asim"
# ds = "sample\indonesia-20230901-20230907.nc"

nch = NCHelper(ds)

reading dataset...
reading dataset done


c:\Users\falbas\miniconda3\Lib\site-packages\xarray\coding\times.py:167: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [43]:
rainc = nch.ds['rainc']
rainsh = nch.ds['rainsh']
rainnc = nch.ds['rainnc']
lev = nch.ds['lev']
lat = nch.ds['lat']
lon = nch.ds['lon']

In [26]:
rainc[0][0].values.max()

0.0

In [ ]:
time = 1
levels = [0, 3, 6, 10, 16]
for l in levels:  
  a = rainc[time][l].values.max()
  b = rainsh[time][l].values.max()
  c = rainnc[time][l].values.max()
  print(lev.values[l])
  print(a)
  print(b)
  print(c)
  print('========')

In [70]:
a = rainc[1][0]
b = rainsh[1][0]
c = rainnc[1][0]

In [71]:
tp = a+b+c

In [73]:
tp.max()

<xarray.DataArray ()>
array(71.78695, dtype=float32)
Coordinates:
    time     datetime64[ns] 2023-11-14T15:00:00
    lev      float64 1e+03

In [46]:
nch.nc2tif(rainc[1][0].values, 'storage/rainc.tif', lat.values, lon.values)
nch.nc2tif(rainnc[1][0].values, 'storage/rainnc.tif', lat.values, lon.values)

In [69]:
nch.nc2tif(c, 'storage/prec.tif', lat.values, lon.values)

In [18]:
lat = nch.ds['lat']
lon = nch.ds['lon']
u = nch.ds['u']
v = nch.ds['v']
tc = nch.ds['tc']

In [47]:
llat = []
for i in lon.values:
  llat.append(lat.values)
llat = np.array(llat)
llat = np.rot90(llat, k=3)

In [49]:
llat[-1][-1]

-15.0001297

In [50]:
output_name = 'lat'
# var = nch.ds[output_name]
var = llat
var = np.flip(var, axis=0)
transform = rasterio.transform.from_bounds(west=lon[0],south=lat[0],east=lon[-1],north=lat[-1],width=lon.shape[0],height=lat.shape[0])
with rasterio.open(
    f"{output_name}.tif",
    "w",
    driver="GTiff",
    height=lat.shape[0],
    width=lon.shape[0],
    count=1,
    dtype=var.dtype,
    crs="+proj=latlong",
    transform=transform,
    nodata=-9999
) as dst:
    dst.write(var, 1)

In [19]:
nch.nc2tif(nch.ds['xlat'][0][0].values, 'xlat', lat.values, lon.values)

In [8]:
nch.nc2rgbtif('tc1', 'tc_color.txt')
nch.nc2rgbtif('tc2', 'tc_color.txt')

generating tc1 rgb geotif images...
output: d:/Kuliah/MBKM/PKKM BMKG/NCHelper/tc1_rgb.tif
generating tc2 rgb geotif images...
output: d:/Kuliah/MBKM/PKKM BMKG/NCHelper/tc2_rgb.tif
